In [1]:
import pandas as pd
import numpy as np
import random

In [3]:

semester_date = "a_20189"
num_students = 50

In [4]:
file = "Exam Sched Prog Datasets.xlsx"
#file = r"C:\Users\William Hazen\Documents\GitHub\exam-timetabling-smac\Exam Sched Prog Datasets.xlsx"
rooms = pd.read_excel(file, sheet_name = "datasets room caps ")
#courses_20221 = pd.read_excel(file, sheet_name = "20221 course size")
#enrolments_20221 = pd.read_excel(file, sheet_name = "20221 anonymized enrolments")
schedule_20221 = pd.read_excel(file, sheet_name = "20221 Final Schedule_fromLSM")

hist_anonymized_enrolments = pd.read_excel(file, sheet_name="hist anonymized enrolments")
#hist_course_size = pd.read_excel(file, sheet_name="hist course size ")

#initalizing
#OG_Course_list = schedule_20221.Course.dropna().values

rooms['Room'] = rooms['Room'].astype(str)
rooms['Room'] = rooms['Bd'].str.cat(rooms['Room'], sep = " ")

#rooms = rooms.drop(['Note','Bd'], axis=1)

# courses_20221['COURSE_CODE'] = courses_20221['COURSE_CODE'].str.cat(courses_20221['SECTIONCD'], sep = "")
# courses_20221 = courses_20221.drop(['SESSION_CD','SECTIONCD','ADMINFACULTYCODE','ADMINDEPT'], axis=1)
# crselist = courses_20221['COURSE_CODE'].tolist()
# sizelist = courses_20221['CURR_REG_QTY'].tolist()
# # Enrolment data
# idlist = enrolments_20221['HASHED_PERSON_ID'].tolist()
# classlist = enrolments_20221['ACAD_ACT_CD'].tolist()

# schedule_20221['Room'] = schedule_20221['Room'].astype(str)
# schedule_20221['Examdate'] = schedule_20221['Examdate'].astype(str)
# schedule_20221['Begin'] = schedule_20221['Begin'].astype(str)
# schedule_20221['Ends'] = schedule_20221['Ends'].astype(str)

# schedule_20221['Room'] = schedule_20221['Bd'].str.cat(schedule_20221['Room'], sep = " ")

# schedule_20221 = schedule_20221.drop(['Bd','Faculty','Department','Section','Sesson','Course Name','Duration'], axis=1)

# schedule_20221.replace('ZZ REFER_TO_FACULTY_SCHEDULE', 'None', inplace=True)
# schedule_20221.replace('ZZ REFER_TO_FACULTY_SCHEDULE', 'None', inplace=True)
# schedule_20221.replace('ZZ ONLIN', 'None', inplace=True)
# schedule_20221.replace('APSCDept APSCDept', 'None', inplace=True)
# schedule_20221.replace('APSCDept ComputerLab', 'None', inplace=True)
# schedule_20221.replace('ZZ KNOX', 'None', inplace=True)
# schedule_20221.replace('ZZ VLAD', 'None', inplace=True)
# schedule_20221.replace('ZZ ONLINMUSIC', 'None', inplace=True)

schedule_20221['Examdate'] = schedule_20221['Examdate'].astype(str)
schedule_20221['Begin'] = schedule_20221['Begin'].astype(str)
examdate_time_list = (schedule_20221['Examdate'].dropna().astype(str) + " : " + schedule_20221['Begin']).dropna().tolist()

ETL = np.unique(examdate_time_list)
examdate_time_list = [i for i in ETL if i != "NaT : nan"]

# course_enrollment_values = courses_enrollments_set.values #.drop(["TEP445H1"], axis=0).values
# sumHe_s = np.sum(courses_enrollments_set, axis=1)

# room_list = schedule_20221['bd_room'].dropna().values
# test_list = room_list.tolist()
# roomlist = [i for i in test_list if i != "None"]

# room_cap_list = schedule_20221['Room Cap'].dropna().values
# test_cap_list = room_cap_list.tolist()
# roomcaplist = [i for i in test_cap_list if i != "None"]


#new processing
rooms['Room'] = rooms['Room'].astype(str)
rooms['Room'] = rooms['Bd'].str.cat(rooms['Room'], sep = " ")

ava_rooms = rooms['Room'].values
ava_room_cap = rooms['Room Cap'].values
hist_anonymized_enrolments['ACAD_ACT_CD'] = hist_anonymized_enrolments['ACAD_ACT_CD'].astype(str)
hist_anonymized_enrolments['ACAD_ACT_CD'] = hist_anonymized_enrolments['ACAD_ACT_CD'].str.cat(hist_anonymized_enrolments['SECTION_CD'])

semeser_list = np.unique(hist_anonymized_enrolments['SESSION_CD']).tolist()
anan_sem = {}
for sem in semeser_list:
    anan_sem["a_{}".format(sem)] = hist_anonymized_enrolments[hist_anonymized_enrolments['SESSION_CD'] == sem]


def processing_fun(semester, num_student):
    name = []
    exam_list = []
    Student_ID_Exams = []
    
    qwe = semester["HASHED_PERSON_ID"].values
    qwer = np.unique(qwe)
    random_sample_names_generator = (qwer[i] for i in range(num_student))

    for i in random_sample_names_generator:
        name.append(i)
        
    Student_ID_List = random.sample(name, len(name))

    for ID in Student_ID_List:
        Student_ID_Exams.append(semester["ACAD_ACT_CD"][semester.eq(ID).any(1)].values)
        
    for exam in Student_ID_Exams:
        exam_list.append(exam.any())
        
    uniq_exams = np.unique(exam_list)
    new_enrol = pd.DataFrame(semester).copy()
    new_enrol["class"] = 1

    temp_df = new_enrol[new_enrol["HASHED_PERSON_ID"].isin(Student_ID_List)]
    final_temp = temp_df[temp_df["ACAD_ACT_CD"].isin(uniq_exams)]

    courses_enrollments_set = pd.pivot_table(data=final_temp, values="class", index="ACAD_ACT_CD", columns=["HASHED_PERSON_ID"], fill_value=0)
    
    return uniq_exams, Student_ID_List, courses_enrollments_set

t_E, t_S, t_C = processing_fun(anan_sem[semester_date], num_students)

#sumHe_s = np.sum(t_C.values, axis=1)

#storing    
exam_set = t_E
student_set = t_S
datetime_slot_set = examdate_time_list
room_set = ava_rooms
room_capacity_set = ava_room_cap
courses_enrollments_set = t_C.values

C:\Users\William Hazen\AppData\Local\Temp\ipykernel_11372\360064934.py:94: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  Student_ID_Exams.append(semester["ACAD_ACT_CD"][semester.eq(ID).any(1)].values)


In [5]:
sumHe_s = np.sum(t_C.values, axis=1)

In [6]:
print("Exam set:" '\n', exam_set,'\n\n', 
      "Course Enrollement Set:" '\n',courses_enrollments_set, '\n\n', 
      "Number of students in course:" '\n',sumHe_s, '\n\n', 
      "Room Capacity set:" '\n',room_capacity_set, '\n\n'
)

Exam set:
 ['AER307H1F' 'APS062H1F' 'APS100H1F' 'APS301H1F' 'APS444H1F' 'APS490Y1Y'
 'APS502H1F' 'CHE204H1F' 'CHE304H1F' 'CHE308H1F' 'CHE353H1F' 'CHE430Y1F'
 'CIV102H1F' 'CIV220H1F' 'CIV416H1F' 'CSC326H1F' 'ECE201H1F' 'ECE212H1F'
 'ECE302H1F' 'ECE470H1F' 'ESC471H1F' 'ESC499Y1Y' 'MAT231H1F' 'MIE230H1F'
 'MIE258H1F' 'MIE270H1F' 'MIE404H1F' 'MIN511H1F' 'PEY500Y1Y'] 

 Course Enrollement Set:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 

 Number of students in course:
 [1 1 6 2 2 2 1 1 1 1 2 1 2 3 1 1 4 5 4 1 1 3 4 1 5 2 4 1 6] 

 Room Capacity set:
 [2000  300   44   74   36   36   35   35   50   50   50   50  231  384
  192  384  192  125   50   50   50   45   45  100  100   50   45  109
  309  335  108  106  120   49   72   50   94   38   38   56   56   53
   53  143   37   37   79   69  235   48   36   48   32   75   60   43
   43   72  110   52   52   52  120   80   82   30   40   30   48   33
   80  112  120

In [7]:
print("Exam Date and time slot:" '\n',examdate_time_list,'\n\n',
      "Room Set:" '\n',room_set, '\n\n', 
      "Student set:" '\n', student_set,'\n\n')

Exam Date and time slot:
 ['2022-04-11 : 09:00:00', '2022-04-11 : 14:00:00', '2022-04-11 : 19:00:00', '2022-04-12 : 09:00:00', '2022-04-12 : 14:00:00', '2022-04-12 : 19:00:00', '2022-04-13 : 09:00:00', '2022-04-13 : 14:00:00', '2022-04-13 : 19:00:00', '2022-04-14 : 09:00:00', '2022-04-14 : 14:00:00', '2022-04-14 : 19:00:00', '2022-04-18 : 09:00:00', '2022-04-18 : 14:00:00', '2022-04-18 : 19:00:00', '2022-04-19 : 09:00:00', '2022-04-19 : 09:30:00', '2022-04-19 : 14:00:00', '2022-04-19 : 18:30:00', '2022-04-19 : 19:00:00', '2022-04-20 : 09:00:00', '2022-04-20 : 09:30:00', '2022-04-20 : 14:00:00', '2022-04-20 : 18:30:00', '2022-04-20 : 19:00:00', '2022-04-21 : 09:00:00', '2022-04-21 : 09:30:00', '2022-04-21 : 14:00:00', '2022-04-21 : 18:30:00', '2022-04-21 : 19:00:00', '2022-04-22 : 09:00:00', '2022-04-22 : 09:30:00', '2022-04-22 : 14:00:00', '2022-04-22 : 18:30:00', '2022-04-22 : 19:00:00', '2022-04-23 : 09:00:00', '2022-04-23 : 09:30:00', '2022-04-23 : 14:00:00', '2022-04-25 : 09:00:00'